In [ ]:
!pip install timm

In [ ]:
import pandas as pd
import numpy as np
import torch
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn as nn
import os
from tqdm import tqdm
import timm
from timm import create_model

In [ ]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

data_transform = transforms.Compose([
    transforms.Resize((224, 224), interpolation=transforms.InterpolationMode.LANCZOS),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

In [ ]:
df = pd.read_csv(r"/kaggle/input/data-split/meta_data_64_16_20.csv")
df

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model = create_model('swin_tiny_patch4_window7_224', pretrained=True, num_classes=3)

model = model.to(device)

In [ ]:
model = nn.DataParallel(model)

state_dict = torch.load("/kaggle/input/swin-tiny/fish_classifier_Swin-Tiny.pth", map_location=device)

model.load_state_dict(state_dict)

model = model.module

model.eval()

In [ ]:
class SwinFeatureExtractor(nn.Module):
    def __init__(self, swin_model):
        super().__init__()
        self.patch_embed = swin_model.patch_embed
        self.layers = swin_model.layers
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        
    def forward(self, x):
        x = self.patch_embed(x)
        x = self.layers(x)
        x = x.permute(0, 3, 1, 2)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        return x
        
extractor = SwinFeatureExtractor(model)
extractor.eval()
extractor.to(device)
extractor

In [ ]:
base_folder = r"/kaggle/input/manhmeo/GG/xzyx7pbr3w-1"

features = []
labels = []
paths = []
types = []

for i, row in tqdm(df.iterrows(), desc="Feature Extraction"):
    path = os.path.join(base_folder, row["Fish Name"], row["Path"])
    try:
        if not os.path.exists(path):
            path = path.replace('_5', '_0')

        image = Image.open(path).convert("RGB")

        tensor_image = data_transform(image).unsqueeze(0).to(device)

        with torch.no_grad():
            features_batch = extractor(tensor_image)
        
        features.append(features_batch.squeeze(0).cpu().numpy())
        labels.append(row["Label"])
        types.append(row["Type"])
        paths.append(os.path.join(row["Fish Name"], row["Path"]))

    except Exception as e:
        print(f"Error at {path}: {e}")

results = pd.DataFrame({"Path": paths, "Label": labels, "Type": types})
results

In [ ]:
cols = [f"Feature_{i}" for i in range(768)]

df1 = pd.DataFrame(features, columns=cols)
df1

In [ ]:
new_df = pd.concat([results, df1], axis=1)
new_df

In [ ]:
new_df.to_csv(r"/kaggle/working/feature_extraction_Swin_Stage4.csv", index=False)